In [1]:
import numpy as np
import pandas as pd

In [2]:
ratings = pd.read_json('./ratings.jsonl', lines=True)
content = pd.read_json('./content.jsonl', lines=True)
ttargets = pd.read_csv('targets.csv', encoding='latin-1', sep=',')

In [3]:
_content = content[['ItemId', 'Actors', 'Director', 'Genre', 'Title', 'imdbVotes']]
_content['Actors'] = _content.Actors.astype(str)
# df_1['plot_keywords'] = df_1.plot_keywords.astype(str) # get keywords from the 'Plot' field
_content['Genre'] = _content.Genre.astype(str)
_content['Director'] = _content.Director.astype(str)


# Removing spaces between names
_content['Director'] = _content['Director'].apply(lambda x: ' '.join(x.replace(' ', '').lower() for x in x.split(', ')))
_content['Actors'] = _content['Actors'].apply(lambda x: ' '.join(x.replace(' ', '').lower() for x in x.split(', ')))

# Discarding the pipes between the plot keywords' and getting only the first five words
# df_1['plot_keywords'] = df_1['plot_keywords'].map(lambda x: x.split('|')) # 'Plot' field but only the keywords
# df_1['plot_keywords'] = df_1['plot_keywords'].apply(lambda x: " ".join(x)) #  'Plot' field but only the keywords

# Discarding the pipes between the genres 
_content['Genre'] = _content['Genre'].map(lambda x: x.lower().replace(', ', ' '))
# _content['Features'] = _content['Actors'] +' '+ _content['Director'] +' '+ _content['Genre']
_content['Features'] = _content['Genre']

_content['imdbVotes'] = _content['imdbVotes'].str.replace('N/A', '1', regex=True)
_content['imdbVotes'] = _content['imdbVotes'].str.replace(',', '', regex=True)
_content['imdbVotes'] = _content['imdbVotes'].astype(str).astype(float)

/tmp/ipykernel_34988/2348077418.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _content['Actors'] = _content.Actors.astype(str)
/tmp/ipykernel_34988/2348077418.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _content['Genre'] = _content.Genre.astype(str)
/tmp/ipykernel_34988/2348077418.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

In [4]:
class Rocchio2:

    def __init__(self, ratings_df):
        self.ratings = ratings_df[['UserId', 'ItemId', 'Rating']].copy()
        self.ratings['Rating'] = self.ratings['Rating'].apply(lambda x: self.rescaleRating(x))

    def fit(self, features_df):
        self.features = self.ratings.copy()

        # Também pode ser compreendido como a computação da matriz de utilizade dos itens.
        self.split_features_into_columns(features_df)

        
        self.compute_user_utility()

    def predict(self, user_item_df):
        targets = user_item_df[['ItemId', 'UserId']].copy()
        targets = targets.merge(self.features, on='ItemId', how='outer')
        targets = targets.fillna(0)

        # return {}
        targets = pd.merge(targets, self.user_utility, on=['UserId'])

        self.similarities = targets[['UserId', 'ItemId']]
        self.similarities['user_DOT_item'] = 0
        self.similarities['user_distance'] = 0
        self.similarities['item_distance'] = 0

        for feat in self.features_list:
            self.similarities['user_DOT_item'] += targets['u_' + feat] * targets[feat]
            self.similarities['user_distance'] += targets['u_' + feat] * targets['u_' + feat]
            self.similarities['item_distance'] += targets[feat]        * targets[feat]

        self.similarities['user_distance'] = self.similarities['user_distance'].pow(1./2)
        self.similarities['item_distance'] = self.similarities['item_distance'].pow(1./2)
        self.similarities['Similarity']    = self.similarities['user_DOT_item'] / (self.similarities['user_distance'] * self.similarities['item_distance'])

        self.similarities = self.similarities.drop(['user_DOT_item', 'user_distance', 'item_distance'], axis=1)

        return self.similarities

    def split_features_into_columns(self, features_df):
        self.features = features_df[['ItemId', 'Features']].copy()
        splitted_features = self.features['Features'].str.split(' ')
        self.features_list = list(dict.fromkeys([y for x in splitted_features for y in x]).keys())

        for feat in self.features_list:
            self.features[feat] = self.features['Features'].apply(lambda x : 1 if feat in x else 0)

        self.features = self.features.drop(['Features'], axis=1)

    def rescaleRating(self, old_rating):
        if old_rating == 10:
            return 3
        elif old_rating > 7:
            return 2
        elif old_rating > 5:
            return 1
        return 0

    def compute_user_utility(self):
        self.user_utility = self.ratings.copy()
        self.user_utility = self.user_utility.merge(self.features, on='ItemId')

        self.user_utility['Frequency'] = self.user_utility.groupby('UserId')['UserId'].transform('count')

        for feat in self.features_list:
            self.user_utility[feat] = self.user_utility[feat] * self.user_utility['Rating'] / self.user_utility['Frequency']

        self.user_utility = self.user_utility.drop(['ItemId', 'Rating', 'Frequency'], axis=1)
        self.user_utility = self.user_utility.groupby('UserId').sum()

        for feat in self.features_list:
            self.user_utility['u_' + feat] = self.user_utility[feat]
            self.user_utility = self.user_utility.drop(feat, axis=1)


In [5]:
similarities = Rocchio2(ratings)
similarities.ratings.head()
similarities.fit(_content[['ItemId', 'Features']])
similarities.features.head()
similarities.user_utility.head()

,u_documentary,u_short,u_news,u_sport,u_horror,u_comedy,u_action,u_adventure,u_drama,u_fantasy,...,u_thriller,u_western,u_animation,u_music,u_musical,u_film-noir,u_adult,u_talk-show,u_reality-tv,u_game-show
UserId,,,,,,,,,,,,,,,,,,,,,
0004d0b59e,0.000000,0.0,0.0,0.0,0.000000,0.000000,2.000000,0.000000,2.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00053f5e11,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00056c20eb,0.000000,0.0,0.0,0.0,0.000000,0.000000,2.000000,2.000000,0.000000,0.000000,...,0.000000,2.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0006246bee,0.018182,0.0,0.0,0.0,0.145455,0.218182,0.527273,0.418182,0.327273,0.145455,...,0.090909,0.036364,0.018182,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0006aabe0b,0.000000,0.0,0.0,0.0,0.000000,0.000000,2.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
result = similarities.predict(ttargets)

/tmp/ipykernel_34988/600581453.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.similarities['user_DOT_item'] = 0
/tmp/ipykernel_34988/600581453.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.similarities['user_distance'] = 0
/tmp/ipykernel_34988/600581453.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

In [7]:
similarities.similarities.head()

,UserId,ItemId,Similarity
0,0006246bee,01d2404d4c,0.301855
1,0006246bee,03d43fdf92,0.862621
2,0006246bee,0808a9666b,0.000000
3,0006246bee,0a5d7dd6f6,0.347133
4,0006246bee,0bab4a8104,0.234140


In [9]:

# result = similarities.predict(ttargets)
result = result.merge(_content[['ItemId', 'imdbVotes']], on='ItemId')
result['Similarity'] = result['Similarity']*result['imdbVotes']
result =result.sort_values(['UserId','Similarity','imdbVotes'], ascending=[True, False, False])
output = result[['UserId', 'ItemId']].copy()

output_file = open('out.csv','w')
print('UserId,ItemId', file=output_file)
pd.set_option("display.max_rows", None, "display.max_columns", None)
output = output.to_numpy()
for user, item in output:
    print('{},{}'.format(user,item), file=output_file)